In [ ]:
!pip install -q -U transformers bitsandbytes
!pip install -q -U peft
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/sem 3 code/GSM_Symbolic_Robustness-main/dataset

/content/drive/MyDrive/sem 3 code/GSM_Symbolic_Robustness-main/dataset


In [ ]:
# %cd /content/GSM_Symbolic_Robustness/dataset

In [ ]:
config = {
    'batch_size': 1,
    'model_name': "google/gemma-2-2b-it",
    'tokenizer_name': 'google/gemma-2-2b-it',
    'token': 'ADD HUGGING FACE TOKEN HERE',
    'is_mistral_or_llama': False,
    'output_name': 'test_output',
    'val_dataset_path': 'gsm_symbolic/data/gsm_names_val.jsonl',
    'train_dataset_path': 'gsm_symbolic/data/gsm_names_train.jsonl', #only need to set this parameter if finetuning
    'num_train_epochs': 250,
}

In [52]:
# import importlib
# importlib.reload(dataset_and_dataloader)

In [53]:
import datasets
import dataset_and_dataloader
from transformers import TrainingArguments
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM
import torch
import json
import pandas as pd
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import re
import numpy as np
from trl import SFTTrainer
from tqdm import tqdm

import wandb
wandb.init(mode='disabled')

In [54]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [55]:
tokenizer = AutoTokenizer.from_pretrained(config['tokenizer_name'], token = config['token'])
if config['is_mistral_or_llama']:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'right'
val_dataset = dataset_and_dataloader.GSM8K_Val_Dataset(config['val_dataset_path'], tokenizer)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=True)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [56]:
train_dataset = Dataset.from_pandas(pd.read_json(config['train_dataset_path'], lines=True))

In [57]:
# load in quantization config
q_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = config["model_name"]

model = AutoModelForCausalLM.from_pretrained(model_name, token=config['token'], quantization_config=q_config)
if config['is_mistral_or_llama']:
  model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [58]:
model.to(device)
model.train()

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm)

In [59]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules= "all-linear",
)
model = get_peft_model(model, lora_config)

In [60]:
def processing(data_point, tokenizer):
    messages = [{'content': data_point["question"], 'role': 'user'}, {'content': data_point["answer"], 'role': 'assistant'}]
    data_point["text"] = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
    return data_point

In [61]:
processed_train_dataset = train_dataset.map(processing, fn_kwargs={"tokenizer": tokenizer}, num_proc = 10, remove_columns = train_dataset.features)

Map (num_proc=10):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [62]:
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()
trainer = SFTTrainer(
    model=model,
    train_dataset=processed_train_dataset,
    dataset_text_field="text",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size = config["batch_size"],
        gradient_accumulation_steps = 4,
        fp16 = True,
        max_steps = config['num_train_epochs'],
        learning_rate = 1e-5,
        logging_steps = 1,
        output_dir = "outputs",
        optim="paged_adamw_8bit",
    ),
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [63]:
trainer.train()

Step,Training Loss
1,2.036300
2,2.450100
3,2.548400
4,2.183600
5,2.099300
6,2.339400
7,2.565800
8,2.174600
9,2.075300
10,2.310500


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67574dcb-7d4c807a70e70d331969e20d;41c53655-4fae-41e6-8c55-03a092dfdacc)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in google/gemma-2-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=250, training_loss=1.281455041885376, metrics={'train_runtime': 491.7238, 'train_samples_per_second': 2.034, 'train_steps_per_second': 0.508, 'total_flos': 2345466939346944.0, 'train_loss': 1.281455041885376, 'epoch': 1.0})

In [64]:
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pr

In [65]:
model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm)

In [66]:
state_dict = model.state_dict()

In [67]:
unexpected_keys = [
    k for k in state_dict if (
        "absmax" in k or
        "quant_map" in k or
        "nested_absmax" in k or
        "nested_quant_map" in k or
        "quant_state" in k
    )
]
for k in unexpected_keys:
    del state_dict[k]

model.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [68]:
torch.save(state_dict, "fine_tuned_gemma.pt")

In [69]:
#free CUDA memory
torch.cuda.empty_cache()
import gc
gc.collect()

51

In [70]:
model.load_state_dict(torch.load("fine_tuned_gemma.pt", weights_only=True))

<All keys matched successfully>